In [1]:
import pandas as pd
from datetime import datetime, timedelta
import requests
import json
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from dotenv import load_dotenv
import os

In [2]:
def make_key():
    return datetime.utcnow().strftime("%Y-%m-%dT%H-%M")
    
def get_recent_data():
    load_dotenv()
    # By latitude and longitude
    # API parameters
    endtime = datetime.utcnow()
    starttime = endtime - timedelta(hours = 1)
    
    options = {}
    options["url"] = "https://www.airnowapi.org/aq/data/"
    options["parameters"] = "OZONE,PM25,PM10,CO,NO2,SO2"
    options["bbox"] = "-180,-90, 180,90"
    options["data_type"] = "B"
    options["format"] = "application/json"
    options["ext"] = "json"
    options["API_KEY"] = os.getenv("API_AIRNOW")
    options['includerawconcentrations'] = "0"
    options["start_date"] = starttime.strftime("%Y-%m-%dT%H")
    options["end_date"] = endtime.strftime("%Y-%m-%dT%H")
    options['monitorType'] = "2"
    options['verbose'] = "0"
    # API request URL
    REQUEST_URL = options["url"] \
                  + "?startDate=" + options["start_date"] \
                  + "&endDate=" + options["end_date"] \
                  + "&parameters=" + options["parameters"] \
                  + "&BBOX=" + options["bbox"] \
                  + "&dataType=" + options["data_type"] \
                  + "&format=" + options["format"] \
                  + "&verbose=" + options['verbose'] \
                + "&monitorType=" + options['monitorType'] \
                  + "&includerawconcentrations=" + options["includerawconcentrations"]\
                  + "&API_KEY=" + options["API_KEY"] 

    session = requests.Session()
    retry = Retry(total=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    r = session.get(REQUEST_URL)
    print(r)
    data = json.loads(r.text)
    df = pd.DataFrame(data)
    return df

def parse_dataframe(df):
    df_parse = pd.DataFrame()
    for parameter in df['Parameter'].unique():
        df_param = df.loc[df['Parameter'] == parameter]
        df_param = df_param.rename({'Unit':f'Unit_{parameter}','Value':f'Value_{parameter}','AQI':f'AQI_{parameter}','Category':f'Category_{parameter}'},axis=1)
        df_param = df_param.drop(labels = 'Parameter',axis=1)
        if len(df_parse) > 0:
            df_parse = df_param.merge(df_parse, on=['Latitude','Longitude','UTC'],how='outer')
        else:
            df_parse = df_param
    return df_parse

def write_to_local(df, key):
    
    tmp_dir = ("./tmp/")
    is_folder = os.path.isdir(tmp_dir)

    if not is_folder:
        os.makedirs(tmp_dir)
    
    filename = './tmp/' + key
    df.to_json(filename)
    return filename

In [3]:
%%time
fn = write_to_local(parse_dataframe(get_recent_data()), make_key())
print(fn)

<Response [200]>
./tmp/2024-04-06T00-38
CPU times: total: 141 ms
Wall time: 6.71 s


In [4]:
pd.read_json(fn).head()

,Latitude,Longitude,UTC,Unit_CO,Value_CO,AQI_CO,Category_CO,Unit_SO2,Value_SO2,AQI_SO2,...,AQI_NO2,Category_NO2,Unit_PM10,Value_PM10,AQI_PM10,Category_PM10,Unit_PM2.5,Value_PM2.5,AQI_PM2.5,Category_PM2.5
0,21.310301,-157.858093,2024-04-05T23:00,PPM,0.2,-999.0,-999.0,PPB,1.0,1.0,...,NaN,NaN,UG/M3,11.0,10.0,1.0,None,NaN,NaN,NaN
1,61.205861,-149.824600,2024-04-05T23:00,PPM,0.2,-999.0,-999.0,None,NaN,NaN,...,NaN,NaN,UG/M3,2.0,2.0,1.0,UG/M3,0.0,0.0,1.0
2,64.845800,-147.727270,2024-04-05T23:00,PPM,0.3,-999.0,-999.0,PPB,0.2,0.0,...,NaN,NaN,UG/M3,6.0,6.0,1.0,UG/M3,3.3,14.0,1.0
3,48.286700,-124.620300,2024-04-05T23:00,PPM,0.1,-999.0,-999.0,PPB,0.2,0.0,...,NaN,NaN,None,NaN,NaN,NaN,UG/M3,1.2,5.0,1.0
4,48.441942,-123.363165,2024-04-05T23:00,PPM,0.1,-999.0,-999.0,PPB,0.7,0.0,...,5.0,1.0,None,NaN,NaN,NaN,UG/M3,3.9,16.0,1.0
